In [47]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(zeallot, warn.conflicts = FALSE)
library(stringr, warn.conflicts = FALSE)
options(repr.plot.width = 9, repr.plot.res = 300)
dataset_id <- "ARPALombardia"
state <- "Lombardia"
source(str_glue("notebooks/integrazioni_regionali/{str_to_lower(state)}/procedures/load_daily.R"))
source("notebooks/integrazioni_regionali/procedure/common_steps.R")
source("notebooks/integrazioni_regionali/procedure/checkpoint.R")
source("notebooks/integrazioni_regionali/procedure/tools.R")
source("src/database/tools.R")
source("src/load/ITA.R")

reference_series <- "a6938c17247455675e4a906599885058"
scia <- open_checkpoint("SCIA", "qc1")$data |>
    filter(station_id == reference_series, variable == "T_MIN") |>
    arrange(date) |>
    collect()

In [48]:
lomb <- open_dataset(file.path(path.lom, "cache")) |>
    filter(original_id == 5922) |>
    arrange(time) |>
    compute()


In [52]:
f0 <- function(lag) {
    daily_min <- lomb |>
        mutate(date = date(time - as.difftime(lag * 60, unit = "secs"))) |>
        group_by(date) |>
        summarise(value = min(value, na.rm = TRUE)) |>
        arrange(date) |>
        collect()
    
    adiffs <- inner_join(scia, daily_min, by = "date") |> mutate(diff = abs(value.x - value.y)) |> pull(diff)
    mean(adiffs < 1e-4, na.rm = TRUE)
}


In [53]:
lags <- tibble(lag = -(6*24):(6*24)) |> mutate(lag_s = lag * 60, f0 = vapply(lag, f0, FUN.VALUE = 0))

In [56]:
lags |> slice_max(f0, n = 60)

lag,lag_s,f0
<int>,<dbl>,<dbl>
1,60,0.7465995
2,120,0.7465995
3,180,0.7465995
4,240,0.7465995
5,300,0.7465995
6,360,0.7465995
7,420,0.7465995
8,480,0.7465995
9,540,0.7465995


In [18]:
library(tsibble)
tl <- lomb |> collect() |> as_tsibble(index = time)

In [22]:
tl$time |> max()

[1] "2015-11-24 11:20:00 CET"